In [1]:
spark

We can call Spark Session Object SparkContext as follows.
* spark.sparkContext
* sc

In [2]:
spark.sparkContext

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
sc
# For backwards compatibility reasons, it’s also still possible to call the SparkContext with sc.

<SparkContext master=local[*] appName=PySparkShell>

## Creating RDD from Collections Using Parallelize Method

In [4]:
dataList =list(range(0, 10))
dataList

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [5]:
dataRdd = sc.parallelize(dataList)
dataRdd

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:489

In [6]:
dataRdd.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [7]:
type(dataRdd)

pyspark.rdd.RDD

### Parallelize Method has two param 
* Collection
* Number of slides

In [8]:
sc.parallelize(dataList, 6).glom().collect()

[[0], [1, 2], [3, 4], [5], [6, 7], [8, 9]]

In [9]:
dataRdd2 = sc.parallelize([2, 5, 3, 1, 7, 9], 2).glom()
sorted(dataRdd2.collect())

[[1, 7, 9], [2, 5, 3]]

In [10]:
keyVal = [('a', 1), ('b', 5), ('c', 7), ('a', 2), ('a', 3), ('b', 4) , ('c', 8)]

In [11]:
keyValRdd = sc.parallelize(keyVal)

## ReduceByKey
* This method is both associative and commutative
* Reduction operator can help break down a task into various partial tasks by calculating partial results which can be used to obtain a final result. 
* It allows certain serial operations to be performed in **parallel**, thereby reducing the number of steps required for certain operations. 
* A reduction operator breaks a serial task into various partial tasks and stores the result into a private copy of the variable. 
* These private copies are then merged into a shared copy at the end.

In [12]:
newKeyVal = keyValRdd.reduceByKey(lambda a, b: a + b).collect()

In [13]:
newKeyVal

[('b', 9), ('c', 15), ('a', 6)]

#### Alternative way 
Both way return the list as w.k.t 
* collect method -> return list
* collectAsMap method -> return map.

In [14]:
from operator import add
newKeyVal = keyValRdd.reduceByKey(add).collect()

## GroupByKey

In [15]:
tupKeyVal = keyValRdd.groupByKey()

In [16]:
list(tupKeyVal.collect()[0][1])

[5, 4]

In [17]:
tupKeyVal.map(lambda para: (para[0], list(para[1]))).collect()

[('b', [5, 4]), ('c', [7, 8]), ('a', [1, 2, 3])]

In [18]:
tupKeyVal.map(lambda para: (para[0], list(para[1]))).collectAsMap()

{'a': [1, 2, 3], 'b': [5, 4], 'c': [7, 8]}

## Fun Programming

In [20]:
from PyFiles import pipeDemo
from PyFiles import pipeDemoFunc

Hello


In [21]:
dataStr = ['james', 'john', 'vin']
dataRddStr = sc.parallelize(dataStr)

In [23]:
pipeRdd = dataRddStr.pipe(pipeDemo)
pipeFuncRdd = dataRddStr.map(lambda x : pipeDemoFunc.fun(x)).collect()

In [24]:
dataRddStr.collect()

['james', 'john', 'vin']

In [25]:
pipeFuncRdd

['Hello...james', 'Hello...john', 'Hello...vin']

In [27]:
pipeRdd.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 16.0 failed 1 times, most recent failure: Lost task 0.0 in stage 16.0 (TID 32, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/ubuntu/ProgramFiles/spark-2.2.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/home/ubuntu/ProgramFiles/spark-2.2.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/ubuntu/ProgramFiles/spark-2.2.1-bin-hadoop2.7/python/pyspark/rdd.py", line 346, in func
    return f(iterator)
  File "/home/ubuntu/ProgramFiles/spark-2.2.1-bin-hadoop2.7/python/pyspark/rdd.py", line 750, in func
    shlex.split(command), env=env, stdin=PIPE, stdout=PIPE)
  File "/hom